<img src="https://storage.googleapis.com/gweb-uniblog-publish-prod/images/gemma-header.width-1200.format-webp.webp" width="100%">

## Instruct Fine-tuning [Gemma](https://blog.google/technology/developers/gemma-open-models/) using qLora and Supervise Finetuning

This is a comprahensive notebook and tutorial on how to fine tune the `gemma-2b-it` Model

All the code will be available on my Github. Do drop by and give a follow and a star⭐.\
[sagarvk24](https://github.com/sagarvk24)
\
[Github Code](https://github.com/sagarvk24/Fine-Tune-LLMs-.git)

Check for GPU (In my case, using Kaggle's T4 x 2)

In [2]:
!nvidia-smi

Tue Feb  4 16:59:40 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   49C    P8              9W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# Step 1: Install the Dependencies

In [3]:
!pip3 install -q -U bitsandbytes==0.42.0 #To load the quantized version of the model
!pip3 install -q -U peft==0.8.2 #For Parameter-Efficient Fine Tuning
!pip3 install -q -U trl==0.7.10 #Supervised Fine Tuning 
!pip3 install -q -U accelerate==0.27.1
!pip3 install -q -U datasets==2.17.0 #for loading the datasets
!pip3 install -q -U transformers==4.38.0 #for main processing and training 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 15.8 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 4.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 3.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.7/115.7 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 6.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 9.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 12.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2023.10.0 which is incompatible.
s3fs 2024.9.0 requires fsspec==2024.9.0.*, but you have fsspec 2023.10.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━

We will load the model using QLoRA Quantization to reduce the usage of memory, as it is very difficult to load such big LLM and fine tune. 

In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, #Quantization
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4", 
    bnb_4bit_compute_dtype=torch.bfloat16 #To avoid information loss while fine tuning
)

# Step 2: Hugging Face: Log In and Access Key Setup 

In [31]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
#In case using Google Colab
#Paste the access token in Colab's Environment 

#import os
#from google.colab import userdata
#os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

# Step 3: Load the Model

In [6]:
#Difference between base model and 'it' model is that, base model is the most raw form of the LLM model. The output is produced in a very raw format, whereas 'it' model produces outputs in a very 

# model_id = "google/gemma-7b-it"
# model_id = "google/gemma-7b"
model_id = "google/gemma-2b-it"
# model_id = "google/gemma-2b"

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [7]:
def get_completion(query: str, model, tokenizer) -> str:
  device = "cuda:0"

  prompt_template = """
  <start_of_turn>user
  Below is an instruction that describes a task. Write a response that appropriately completes the request.
  {query}
  <end_of_turn>\n<start_of_turn>model
  

  """
  prompt = prompt_template.format(query=query)

  encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)

  model_inputs = encodeds.to(device)


  generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)
  # decoded = tokenizer.batch_decode(generated_ids)
  decoded = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
  return (decoded)

### Testing 

In [8]:
result = get_completion(query="code to check a palindrome number in C++", model=model, tokenizer=tokenizer)
print(result)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.



  user
  Below is an instruction that describes a task. Write a response that appropriately completes the request.
  code to check a palindrome number in C++
  
model
  

    Sure. The following code in C++ checks if a number is a palindrome number:

```c++
#include <iostream>
#include <string>
using namespace std;

bool isPalindrome(int num) {
    int reversed = 0;
    int original = num;
    while (original != 0) {
        int digit = original % 10;
        reversed = reversed * 10 + digit;
        original /= 10;
    }
    return original == reversed;
}

int main() {
    int num = 1212;
    if (isPalindrome(num)) {
        cout << num << " is a palindrome number" << endl;
    } else {
        cout << num << " is not a palindrome number" << endl;
    }
    return 0;
}
```


# Step 4: Load the Dataset

In [9]:
from datasets import load_dataset

dataset = load_dataset("TokenBender/code_instructions_122k_alpaca_style", split="train")
dataset

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['input', 'instruction', 'text', 'output'],
    num_rows: 121959
})

In [10]:
df = dataset.to_pandas()
df.head(10)

,input,instruction,text,output
0,"[1, 2, 3, 4, 5]",Create a function to calculate the sum of a se...,Below is an instruction that describes a task....,# Python code\ndef sum_sequence(sequence):\n ...
1,"str1 = ""Hello ""\nstr2 = ""world""",Develop a function that will add two strings,Below is an instruction that describes a task....,"def add_strings(str1, str2):\n """"""This func..."
2,,Design a data structure in C++ to store inform...,Below is an instruction that describes a task....,#include <map>\n#include <string>\n\nclass Gro...
3,"[3, 1, 4, 5, 9, 0]",Implement a sorting algorithm to sort a given ...,Below is an instruction that describes a task....,def bubble_sort(arr):\n n = len(arr)\n \n ...
4,Not applicable,Design a Swift application for tracking expens...,Below is an instruction that describes a task....,import UIKit\n\nclass ExpenseViewController: U...
5,Not Applicable,Create a REST API to convert a UNIX timestamp ...,Below is an instruction that describes a task....,<?php\n$timestamp = $_GET['timestamp'];\n\nif(...
6,website: www.example.com \ndata to crawl: phon...,Generate a Python code for crawling a website ...,Below is an instruction that describes a task....,import requests\nimport re\n\ndef crawl_websit...
7,,Create a Python list comprehension to get the ...,Below is an instruction that describes a task....,"[x*x for x in [1, 2, 3, 5, 8, 13]]"
8,,Create a MySQL query to find the most expensiv...,Below is an instruction that describes a task....,SELECT * FROM products ORDER BY price DESC LIM...
9,Not applicable,Create a data structure in Java for storing an...,Below is an instruction that describes a task....,public class Library {\n \n // map of books in...


# Step 5: Format the Dataset

In [11]:
def generate_prompt(data_point):
    """Gen. input text based on a prompt, task instruction, (context info.), and answer

    :param data_point: dict: Data point
    :return: dict: tokenzed prompt
    """
    prefix_text = 'Below is an instruction that describes a task. Write a response that ' \
               'appropriately completes the request.\n\n'
    # Samples with additional context into.
    if data_point['input']:
        text = f"""<start_of_turn>user {prefix_text} {data_point["instruction"]} here are the inputs {data_point["input"]} <end_of_turn>\n<start_of_turn>model{data_point["output"]} <end_of_turn>"""
    # Without
    else:
        text = f"""<start_of_turn>user {prefix_text} {data_point["instruction"]} <end_of_turn>\n<start_of_turn>model{data_point["output"]} <end_of_turn>"""
    return text

# add the "prompt" column in the dataset
text_column = [generate_prompt(data_point) for data_point in dataset]
dataset = dataset.add_column("prompt", text_column)

## Step 5.1: Tokenize the data to make it easier for model to understand

In [12]:
dataset = dataset.shuffle(seed=1234)  # Shuffle dataset here
dataset = dataset.map(lambda samples: tokenizer(samples["prompt"]), batched=True)

Map:   0%|          | 0/121959 [00:00<?, ? examples/s]

## Step 5.2: Split the Dataset

In [13]:
dataset = dataset.train_test_split(test_size=0.2)
train_data = dataset["train"]
test_data = dataset["test"]

In [14]:
print(test_data)

Dataset({
    features: ['input', 'instruction', 'text', 'output', 'prompt', 'input_ids', 'attention_mask'],
    num_rows: 24392
})


# Step 6: Apply LoRA (Low Rank Adaptation)
Magic with peft! 
-> Load a PeftModel and specify that we are going to use low-rank adapters (LoRA) using get_peft_model utility function and  the prepare_model_for_kbit_training method from PEFT.

In [15]:
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [16]:
print(model)

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear4bit(in_features=16384, out_features=2048, bias=False)
          (act_fn): GELUActivation()
        )
        (input_layernorm): GemmaRMSNorm()
        (post_attention_layernorm): GemmaRMSNorm()
     

In [17]:
#to determine all the linear layerss while using bitsandbytes
import bitsandbytes as bnb
def find_all_linear_names(model):
  cls = bnb.nn.Linear4bit #if args.bits == 4 else (bnb.nn.Linear8bitLt if args.bits == 8 else torch.nn.Linear)
  lora_module_names = set()
  for name, module in model.named_modules():
    if isinstance(module, cls):
      names = name.split('.')
      lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names: # needed for 16-bit
      lora_module_names.remove('lm_head')
  return list(lora_module_names)

In [18]:
modules = find_all_linear_names(model)
print(modules)

['down_proj', 'q_proj', 'gate_proj', 'k_proj', 'o_proj', 'v_proj', 'up_proj']


In [21]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=32, #Adjust according to your resources
    lora_alpha=16,
    target_modules=modules,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

In [22]:
trainable, total = model.get_nb_trainable_parameters()
print(f"Trainable: {trainable} | total: {total} | Percentage: {trainable/total*100:.4f}%")

Trainable: 39223296 | total: 2545395712 | Percentage: 1.5410%


# Step 7: Setup for the Training

## Fine-Tuning with QLoRA and Supervised Fine-Tuning using SFTT Trainer

In [49]:
import transformers
from trl import SFTTrainer

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"  # Fix for padding side warning
torch.cuda.empty_cache()

trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=test_data,
    dataset_text_field="prompt",
    peft_config=lora_config,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=2,  # Increased Batch size
        gradient_accumulation_steps=2,  # Adjusted accumulation
        warmup_ratio=0.03,  
        max_steps=10, #Adjust accordingly
        learning_rate=1e-4,  # Stable LR
        logging_steps=1,
        output_dir="outputs",
        optim="adamw_bnb_8bit",  # Efficient optimizer
        save_strategy="epoch",
        fp16=True,  # Enable mixed precision for speedup
        report_to = "none" #can change it to "wandb" to log results there
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)


In [39]:
import torch
print(torch.cuda.is_available())  # Should return True
print(torch.cuda.device_count())  # Should return 2 (since you have T4 x2)

True
2


In [63]:
torch.cuda.empty_cache()

# Step 8: Start the Training!😇❤️

In [41]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [43]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

When working on Kaggle, in case you wish to start the training again, with changed parameters, then run the command first (added just below)

In [51]:
#rm -rf outputs/checkpoint-10

In [52]:
model.config.use_cache = False  # Disable cache for training
trainer.train()  # Start fine-tuning

Step,Training Loss
1,1.450600
2,1.352900
3,1.053300
4,1.039100
5,1.332700
6,1.344200
7,1.344200
8,1.210900
9,1.088800
10,1.303700


TrainOutput(global_step=10, training_loss=1.2520320177078248, metrics={'train_runtime': 185.9013, 'train_samples_per_second': 0.43, 'train_steps_per_second': 0.054, 'total_flos': 278185265479680.0, 'train_loss': 1.2520320177078248, 'epoch': 0.0})

# Step 9: Share the Adapters on the 🤗 Hub! 

In [57]:
new_model = "gemma-2B-Instruct-Finetune-Sagar" #Name of the model you will be pushing to huggingface model hub

In [58]:
trainer.model.save_pretrained(new_model)

In [59]:
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
)
merged_model= PeftModel.from_pretrained(base_model, new_model)
merged_model= merged_model.merge_and_unload()

# Save the merged model
merged_model.save_pretrained("merged_model",safe_serialization=True)
tokenizer.save_pretrained("merged_model")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [60]:
# Push the model and tokenizer to the Hugging Face Model Hub
merged_model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/sagarvk24/gemma-2B-Instruct-Finetune-Sagar/commit/8c8b4074255d4a7cbdaa4064b2d841fa629bea5f', commit_message='Upload tokenizer', commit_description='', oid='8c8b4074255d4a7cbdaa4064b2d841fa629bea5f', pr_url=None, repo_url=RepoUrl('https://huggingface.co/sagarvk24/gemma-2B-Instruct-Finetune-Sagar', endpoint='https://huggingface.co', repo_type='model', repo_id='sagarvk24/gemma-2B-Instruct-Finetune-Sagar'), pr_revision=None, pr_num=None)

### To avoid Warnings

In [61]:
tokenizer.padding_side = 'left'
tokenizer.pad_token = tokenizer.eos_token

# Step 10: Test on Fine-Tuned Model🔎

In [74]:
# import torch

# # Empty the CUDA cache before inference
# torch.cuda.empty_cache()

# # Adjust the max tokens (reduce for memory efficiency)
# max_new_tokens = 50  # You can adjust this value based on your GPU memory

# # Use mixed precision (FP16) if supported by your model
# from torch.amp import autocast

# # Modify the inference call with proper generation parameters
# def get_completion(query, model, tokenizer):
#     # Tokenize the input query
#     inputs = tokenizer(query, return_tensors="pt")

#     # Move the inputs to the GPU
#     input_ids = inputs["input_ids"].to(model.device)
    
#     # Perform inference with mixed precision
#     with torch.no_grad():  # No gradients needed for inference
#         with autocast(device_type='cuda'):  # Specify 'cuda' for mixed precision
#             generated_ids = model.generate(input_ids=input_ids, 
#                                            max_new_tokens=max_new_tokens, 
#                                            do_sample=True, 
#                                            pad_token_id=tokenizer.eos_token_id)
    
#     # Decode and return the result
#     return tokenizer.decode(generated_ids[0], skip_special_tokens=True)

# # Run the function
# result = get_completion(query="code to print Hello in Python", model=merged_model, tokenizer=tokenizer)
# print(result)


In [76]:
# result = get_completion(query="code the fibonacci series in python using reccursion", model=merged_model, tokenizer=tokenizer)
# print(result)